# Simple Analyzer for Leak Detector

In [1]:
import pandas as pd


## Convert CSV to Dataframe

In [4]:
extractedDF = pd.read_csv('./leak_raw_231011.csv')
extractedDF

,init_url,final_url,req_url,req_method,ref_data,post_data,url_leaks,post_leaks,referer_leaks
0,http://wheatonarts.org/,https://www.wheatonarts.org/,http://wheatonarts.org/,GET,NaN,NaN,[],[],[]
1,http://wheatonarts.org/,https://www.wheatonarts.org/,https://www.wheatonarts.org/,GET,NaN,NaN,[],[],[]
2,http://wheatonarts.org/,https://www.wheatonarts.org/,https://fonts.googleapis.com/css?family=Open+S...,GET,https://www.wheatonarts.org/,NaN,[],[],[]
3,http://wheatonarts.org/,https://www.wheatonarts.org/,https://www.wheatonarts.org/wp-content/themes/...,GET,https://www.wheatonarts.org/,NaN,[],[],[]
4,http://wheatonarts.org/,https://www.wheatonarts.org/,https://www.wheatonarts.org/wp-content/themes/...,GET,https://www.wheatonarts.org/,NaN,[],[],[]
...,...,...,...,...,...,...,...,...,...
79825,http://osheaga.com/,https://osheaga.com/fr,https://www.gstatic.com/eureka/clank/93/cast_s...,GET,https://www.youtube.com/,NaN,[],[],[]
79826,http://osheaga.com/,https://osheaga.com/fr,https://www.facebook.com/tr/?id=38692092893660...,GET,https://osheaga.com/,NaN,"[('urlencode', '/fr'), ('urlencode', 'https://...",[],[]
79827,http://osheaga.com/,https://osheaga.com/fr,wss://ws.hotjar.com/api/v2/client/ws?v=4,NaN,NaN,"{""sid"":""6fe607cf-45ed-46ba-8956-f8be89ef6d7c"",...",[],"[('https://osheaga.com/fr',), ('/fr',), ('oshe...",[]
79828,http://osheaga.com/,https://osheaga.com/fr,wss://ws.hotjar.com/api/v2/client/ws?v=4,NaN,NaN,"{""sid"":""6fe607cf-45ed-46ba-8956-f8be89ef6d7c"",...",[],[],[]


## Leaks on Referer

We still need to compare with the referrer policy to make it accurate

In [5]:
# exclude [] and empty string

ref_leak = extractedDF[(extractedDF['referer_leaks'] != '[]') & (extractedDF['referer_leaks'] != 'None')]
ref_leak = ref_leak[ref_leak['referer_leaks'].notnull()]
ref_leak

,init_url,final_url,req_url,req_method,ref_data,post_data,url_leaks,post_leaks,referer_leaks
208,http://virtuaoperator.pl/,https://www.virtuaoperator.pl/,https://www.google.com/recaptcha/api2/webworke...,GET,https://www.google.com/recaptcha/api2/anchor?a...,NaN,[],[],"[('urlencode', 'base64', 'www.virtuaoperator.p..."
210,http://virtuaoperator.pl/,https://www.virtuaoperator.pl/,https://www.google.com/recaptcha/api2/reload?k...,POST,https://www.google.com/recaptcha/api2/anchor?a...,\nlLirU0na9roYU3wDDisGJEVTù\n03AFcWeA4mfv4J0...,[],[],"[('urlencode', 'base64', 'www.virtuaoperator.p..."
480,http://farsondigitalwatercams.com/,https://www.farsondigitalwatercams.com/,https://googleads.g.doubleclick.net/pagead/adv...,GET,https://googleads.g.doubleclick.net/pagead/ads...,NaN,[],[],"[('www.farsondigitalwatercams.com',), ('urlenc..."
848,http://clackesd.k12.or.us/,https://www.clackesd.org/,https://www.facebook.com/platform/plugin/tab/r...,GET,https://www.facebook.com/v14.0/plugins/page.ph...,NaN,"[('www.clackesd.org',), ('urlencode', 'https:/...",[],"[('www.clackesd.org',), ('urlencode', 'www.cla..."
849,http://clackesd.k12.or.us/,https://www.clackesd.org/,https://www.facebook.com/platform/plugin/page/...,POST,https://www.facebook.com/v14.0/plugins/page.ph...,event_name=page_plugin.tab.configured&page_id=...,[],"[('www.clackesd.org',), ('urlencode', 'https:/...","[('www.clackesd.org',), ('urlencode', 'www.cla..."
...,...,...,...,...,...,...,...,...,...
74822,http://veganinthefreezer.com/,https://veganinthefreezer.com/,https://app.grow.me/polyfills.js,GET,https://app.grow.me/register?callback=https%3A...,NaN,[],[],"[('urlencode', 'https://veganinthefreezer.com/..."
74825,http://veganinthefreezer.com/,https://veganinthefreezer.com/,https://app.grow.me/main.js,GET,https://app.grow.me/register?callback=https%3A...,NaN,[],[],"[('urlencode', 'https://veganinthefreezer.com/..."
74827,http://veganinthefreezer.com/,https://veganinthefreezer.com/,https://app.grow.me/static/js/app.6.61.0.js,GET,https://app.grow.me/register?callback=https%3A...,NaN,[],[],"[('urlencode', 'https://veganinthefreezer.com/..."
77427,http://headstuff.org/,https://headstuff.org/,https://gum.criteo.com/sid/json?origin=publish...,GET,https://gum.criteo.com/syncframe?origin=publis...,NaN,"[('headstuff.org',), ('urlencode', 'headstuff....",[],"[('headstuff.org',), ('urlencode', 'headstuff...."


In [8]:
# drop unnecessary columns and print to csv

ref_leak_print =  ref_leak.drop(['url_leaks', 'post_leaks', 'post_data'], axis=1)
ref_leak_print.to_csv("leak_ref_print.csv", index=False)


,init_url,final_url,req_url,req_method,ref_data,referer_leaks
208,http://virtuaoperator.pl/,https://www.virtuaoperator.pl/,https://www.google.com/recaptcha/api2/webworke...,GET,https://www.google.com/recaptcha/api2/anchor?a...,"[('urlencode', 'base64', 'www.virtuaoperator.p..."
210,http://virtuaoperator.pl/,https://www.virtuaoperator.pl/,https://www.google.com/recaptcha/api2/reload?k...,POST,https://www.google.com/recaptcha/api2/anchor?a...,"[('urlencode', 'base64', 'www.virtuaoperator.p..."
480,http://farsondigitalwatercams.com/,https://www.farsondigitalwatercams.com/,https://googleads.g.doubleclick.net/pagead/adv...,GET,https://googleads.g.doubleclick.net/pagead/ads...,"[('www.farsondigitalwatercams.com',), ('urlenc..."
848,http://clackesd.k12.or.us/,https://www.clackesd.org/,https://www.facebook.com/platform/plugin/tab/r...,GET,https://www.facebook.com/v14.0/plugins/page.ph...,"[('www.clackesd.org',), ('urlencode', 'www.cla..."
849,http://clackesd.k12.or.us/,https://www.clackesd.org/,https://www.facebook.com/platform/plugin/page/...,POST,https://www.facebook.com/v14.0/plugins/page.ph...,"[('www.clackesd.org',), ('urlencode', 'www.cla..."
...,...,...,...,...,...,...
74822,http://veganinthefreezer.com/,https://veganinthefreezer.com/,https://app.grow.me/polyfills.js,GET,https://app.grow.me/register?callback=https%3A...,"[('urlencode', 'https://veganinthefreezer.com/..."
74825,http://veganinthefreezer.com/,https://veganinthefreezer.com/,https://app.grow.me/main.js,GET,https://app.grow.me/register?callback=https%3A...,"[('urlencode', 'https://veganinthefreezer.com/..."
74827,http://veganinthefreezer.com/,https://veganinthefreezer.com/,https://app.grow.me/static/js/app.6.61.0.js,GET,https://app.grow.me/register?callback=https%3A...,"[('urlencode', 'https://veganinthefreezer.com/..."
77427,http://headstuff.org/,https://headstuff.org/,https://gum.criteo.com/sid/json?origin=publish...,GET,https://gum.criteo.com/syncframe?origin=publis...,"[('headstuff.org',), ('urlencode', 'headstuff...."


### Google

In [9]:
# list of referer leaks to google
ref_leak_print[ref_leak_print['req_url'].str.contains('google')]

,init_url,final_url,req_url,req_method,ref_data,post_data,url_leaks,post_leaks,referer_leaks
208,http://virtuaoperator.pl/,https://www.virtuaoperator.pl/,https://www.google.com/recaptcha/api2/webworke...,GET,https://www.google.com/recaptcha/api2/anchor?a...,NaN,[],[],"[('urlencode', 'base64', 'www.virtuaoperator.p..."
210,http://virtuaoperator.pl/,https://www.virtuaoperator.pl/,https://www.google.com/recaptcha/api2/reload?k...,POST,https://www.google.com/recaptcha/api2/anchor?a...,\nlLirU0na9roYU3wDDisGJEVTù\n03AFcWeA4mfv4J0...,[],[],"[('urlencode', 'base64', 'www.virtuaoperator.p..."
480,http://farsondigitalwatercams.com/,https://www.farsondigitalwatercams.com/,https://googleads.g.doubleclick.net/pagead/adv...,GET,https://googleads.g.doubleclick.net/pagead/ads...,NaN,[],[],"[('www.farsondigitalwatercams.com',), ('urlenc..."
1185,http://bpmob.com/,https://bpmobile.com/,https://www.google.com/recaptcha/api2/webworke...,GET,https://www.google.com/recaptcha/api2/anchor?a...,NaN,[],[],"[('base64', 'bpmobile.com'), ('urlencode', 'ba..."
1190,http://bpmob.com/,https://bpmobile.com/,https://www.google.com/recaptcha/api2/reload?k...,POST,https://www.google.com/recaptcha/api2/anchor?a...,\nMydHw_zggsxIJuhSbyOmPv5Rù\n03AFcWeA7B1k-4x...,[],[],"[('base64', 'bpmobile.com'), ('urlencode', 'ba..."
...,...,...,...,...,...,...,...,...,...
69110,http://dionice.net/,http://dionice.net/forum/,https://googleads.g.doubleclick.net/pagead/adv...,GET,https://googleads.g.doubleclick.net/pagead/ads...,NaN,[],[],"[('dionice.net',), ('urlencode', 'dionice.net'..."
72675,http://postman-echo.com/,https://www.postman.com/postman/workspace/publ...,https://accounts.google.com/gsi/client,GET,https://www.postman.com/postman/workspace/publ...,NaN,[],[],"[('ctx=documentation',), ('/postman/workspace/..."
72694,http://postman-echo.com/,https://www.postman.com/postman/workspace/publ...,https://accounts.google.com/gsi/style,GET,https://www.postman.com/postman/workspace/publ...,NaN,[],[],"[('ctx=documentation',), ('/postman/workspace/..."
72695,http://postman-echo.com/,https://www.postman.com/postman/workspace/publ...,https://accounts.google.com/gsi/status?client_...,GET,https://www.postman.com/postman/workspace/publ...,NaN,[],[],"[('ctx=documentation',), ('/postman/workspace/..."


In [12]:
# counting on referer leaks to google
ref_leak_print[ref_leak_print['req_url'].str.contains('google')].count()

init_url         82
final_url        82
req_url          82
req_method       82
ref_data         82
post_data         8
url_leaks        82
post_leaks       82
referer_leaks    82
dtype: int64

### META / Facebook

In [15]:
# list of referer leaks to Meta or Facebook
ref_leak_print[ref_leak_print['req_url'].str.contains('meta') | ref_leak_print['req_url'].str.contains('facebook')]

,init_url,final_url,req_url,req_method,ref_data,referer_leaks
848,http://clackesd.k12.or.us/,https://www.clackesd.org/,https://www.facebook.com/platform/plugin/tab/r...,GET,https://www.facebook.com/v14.0/plugins/page.ph...,"[('www.clackesd.org',), ('urlencode', 'www.cla..."
849,http://clackesd.k12.or.us/,https://www.clackesd.org/,https://www.facebook.com/platform/plugin/page/...,POST,https://www.facebook.com/v14.0/plugins/page.ph...,"[('www.clackesd.org',), ('urlencode', 'www.cla..."
882,http://clackesd.k12.or.us/,https://www.clackesd.org/,https://www.facebook.com/login/?next=https%3A%...,GET,https://www.facebook.com/v14.0/plugins/page.ph...,"[('www.clackesd.org',), ('urlencode', 'www.cla..."
889,http://clackesd.k12.or.us/,https://www.clackesd.org/,https://www.facebook.com/ajax/bz?__a=1&__ccg=E...,POST,https://www.facebook.com/v14.0/plugins/page.ph...,"[('www.clackesd.org',), ('urlencode', 'www.cla..."
10089,http://vhnd.com/,https://www.vhnd.com/,https://www.facebook.com/pages/call_to_action/...,POST,https://www.facebook.com/plugins/page.php?adap...,"[('www.vhnd.com',), ('urlencode', 'www.vhnd.co..."
10094,http://vhnd.com/,https://www.vhnd.com/,https://www.facebook.com/login/?next=https%3A%...,GET,https://www.facebook.com/plugins/page.php?adap...,"[('www.vhnd.com',), ('urlencode', 'www.vhnd.co..."
10097,http://vhnd.com/,https://www.vhnd.com/,https://www.facebook.com/ajax/bz?__a=1&__ccg=E...,POST,https://www.facebook.com/plugins/page.php?adap...,"[('www.vhnd.com',), ('urlencode', 'www.vhnd.co..."
10115,http://vhnd.com/,https://www.vhnd.com/,https://www.facebook.com/pages/call_to_action/...,POST,https://www.facebook.com/plugins/page.php?adap...,"[('www.vhnd.com',), ('urlencode', 'www.vhnd.co..."
10121,http://vhnd.com/,https://www.vhnd.com/,https://www.facebook.com/login/?next=https%3A%...,GET,https://www.facebook.com/plugins/page.php?adap...,"[('www.vhnd.com',), ('urlencode', 'www.vhnd.co..."
10124,http://vhnd.com/,https://www.vhnd.com/,https://www.facebook.com/ajax/bz?__a=1&__ccg=E...,POST,https://www.facebook.com/plugins/page.php?adap...,"[('www.vhnd.com',), ('urlencode', 'www.vhnd.co..."


In [16]:
# Counting on referer leaks to Meta or Facebook
ref_leak_print[ref_leak_print['req_url'].str.contains('meta') | ref_leak_print['req_url'].str.contains('facebook')].count()

init_url         19
final_url        19
req_url          19
req_method       19
ref_data         19
referer_leaks    19
dtype: int64

### Tiktok

In [55]:
# list of referer leaks to tiktok
ref_leak_print[ref_leak_print['req_url'].str.contains('tiktok')] 

,init_url,final_url,req_url,req_method,post_data,post_leaks


In [36]:
# Counting on referer leaks to tiktok
ref_leak_print[(ref_leak_print['req_url'].str.contains('tiktok') | ref_leak_print['req_url'].str.contains('pixel')) & ~(ref_leak_print['req_url'].str.contains('google'))].count()

init_url         7
final_url        7
req_url          7
req_method       7
ref_data         7
referer_leaks    7
dtype: int64

## Leaks on URL

In [28]:
# exclude [] and empty string

url_leak = extractedDF[(extractedDF['url_leaks'] != '[]') & (extractedDF['url_leaks'] != 'None')]
url_leak = url_leak[url_leak['url_leaks'].notnull()]
url_leak

,init_url,final_url,req_url,req_method,ref_data,post_data,url_leaks,post_leaks,referer_leaks
122,http://wheatonarts.org/,https://www.wheatonarts.org/,https://analytics.google.com/g/collect?v=2&tid...,POST,https://www.wheatonarts.org/,NaN,"[('www.wheatonarts.org',), ('urlencode', 'http...",None,[]
201,http://virtuaoperator.pl/,https://www.virtuaoperator.pl/,https://www.google.com/recaptcha/api2/anchor?a...,GET,https://www.virtuaoperator.pl/,NaN,"[('urlencode', 'base64', 'www.virtuaoperator.p...",[],[]
331,http://retailgators.com/,https://www.retailgators.com/,https://xbytedev.co/xbyte-analytics/matomo.php...,POST,https://www.retailgators.com/,NaN,"[('urlencode', 'https://www.retailgators.com/'...",None,[]
336,http://retailgators.com/,https://www.retailgators.com/,wss://vts.zohopublic.com/watchws?x-e=embed1.xb...,NaN,NaN,NaN,"[('urlencode', 'https://www.retailgators.com/'...",None,[]
341,http://retailgators.com/,https://www.retailgators.com/,https://www.google-analytics.com/j/collect?v=1...,POST,https://www.retailgators.com/,NaN,"[('urlencode', 'https://www.retailgators.com/'...",None,[]
...,...,...,...,...,...,...,...,...,...
79811,http://osheaga.com/,https://osheaga.com/fr,https://www.facebook.com/tr/?id=16057439283596...,GET,https://osheaga.com/,NaN,"[('urlencode', '/fr'), ('urlencode', 'https://...",[],[]
79812,http://osheaga.com/,https://osheaga.com/fr,https://adservice.google.com/ddm/fls/z/dc_pre=...,GET,https://osheaga.com/,NaN,"[('urlencode', '/fr'), ('urlencode', 'https://...",[],[]
79818,http://osheaga.com/,https://osheaga.com/fr,https://connect.facebook.net/signals/config/38...,GET,https://osheaga.com/,NaN,"[('urlencode', 'osheaga.com'), ('osheaga.com',)]",[],[]
79819,http://osheaga.com/,https://osheaga.com/fr,https://www.facebook.com/tr/?id=19916247543847...,GET,https://osheaga.com/,NaN,"[('urlencode', '/fr'), ('urlencode', 'https://...",[],[]


In [31]:
# drop unnecessary columns and print to csv

url_leak_print =  url_leak.drop(['post_leaks', 'ref_data', 'referer_leaks', 'post_data'], axis=1)
url_leak_print.to_csv("url_print.csv", index=False)
url_leak_print

,init_url,final_url,req_url,req_method,url_leaks
122,http://wheatonarts.org/,https://www.wheatonarts.org/,https://analytics.google.com/g/collect?v=2&tid...,POST,"[('www.wheatonarts.org',), ('urlencode', 'http..."
201,http://virtuaoperator.pl/,https://www.virtuaoperator.pl/,https://www.google.com/recaptcha/api2/anchor?a...,GET,"[('urlencode', 'base64', 'www.virtuaoperator.p..."
331,http://retailgators.com/,https://www.retailgators.com/,https://xbytedev.co/xbyte-analytics/matomo.php...,POST,"[('urlencode', 'https://www.retailgators.com/'..."
336,http://retailgators.com/,https://www.retailgators.com/,wss://vts.zohopublic.com/watchws?x-e=embed1.xb...,NaN,"[('urlencode', 'https://www.retailgators.com/'..."
341,http://retailgators.com/,https://www.retailgators.com/,https://www.google-analytics.com/j/collect?v=1...,POST,"[('urlencode', 'https://www.retailgators.com/'..."
...,...,...,...,...,...
79811,http://osheaga.com/,https://osheaga.com/fr,https://www.facebook.com/tr/?id=16057439283596...,GET,"[('urlencode', '/fr'), ('urlencode', 'https://..."
79812,http://osheaga.com/,https://osheaga.com/fr,https://adservice.google.com/ddm/fls/z/dc_pre=...,GET,"[('urlencode', '/fr'), ('urlencode', 'https://..."
79818,http://osheaga.com/,https://osheaga.com/fr,https://connect.facebook.net/signals/config/38...,GET,"[('urlencode', 'osheaga.com'), ('osheaga.com',)]"
79819,http://osheaga.com/,https://osheaga.com/fr,https://www.facebook.com/tr/?id=19916247543847...,GET,"[('urlencode', '/fr'), ('urlencode', 'https://..."


### GOOGLE

In [34]:
# list of url leaks to google
url_leak_print[url_leak_print['req_url'].str.contains('google')]

,init_url,final_url,req_url,req_method,url_leaks
122,http://wheatonarts.org/,https://www.wheatonarts.org/,https://analytics.google.com/g/collect?v=2&tid...,POST,"[('www.wheatonarts.org',), ('urlencode', 'http..."
201,http://virtuaoperator.pl/,https://www.virtuaoperator.pl/,https://www.google.com/recaptcha/api2/anchor?a...,GET,"[('urlencode', 'base64', 'www.virtuaoperator.p..."
341,http://retailgators.com/,https://www.retailgators.com/,https://www.google-analytics.com/j/collect?v=1...,POST,"[('urlencode', 'https://www.retailgators.com/'..."
345,http://retailgators.com/,https://www.retailgators.com/,https://www.google-analytics.com/g/collect?v=2...,POST,"[('urlencode', 'https://www.retailgators.com/'..."
399,http://farsondigitalwatercams.com/,https://www.farsondigitalwatercams.com/,https://analytics.google.com/g/collect?v=2&tid...,POST,"[('www.farsondigitalwatercams.com',), ('urlenc..."
...,...,...,...,...,...
79771,http://osheaga.com/,https://osheaga.com/fr,https://googleads.g.doubleclick.net/pagead/vie...,GET,"[('urlencode', '/fr'), ('urlencode', 'https://..."
79785,http://osheaga.com/,https://osheaga.com/fr,https://www.google-analytics.com/j/collect?v=1...,POST,"[('urlencode', '/fr'), ('urlencode', 'https://..."
79786,http://osheaga.com/,https://osheaga.com/fr,https://www.google.com/pagead/1p-user-list/AW-...,GET,"[('urlencode', '/fr'), ('urlencode', 'https://..."
79794,http://osheaga.com/,https://osheaga.com/fr,https://analytics.google.com/g/collect?v=2&tid...,POST,"[('urlencode', '/fr'), ('urlencode', 'https://..."


### Facebook or Meta

In [35]:
# list of url leaks to Meta or Facebook
url_leak_print[url_leak_print['req_url'].str.contains('meta') | url_leak_print['req_url'].str.contains('facebook')]

,init_url,final_url,req_url,req_method,url_leaks
611,http://limonhost.net/,https://www.limonhost.net/,https://connect.facebook.net/signals/config/79...,GET,"[('urlencode', 'www.limonhost.net'), ('www.lim..."
618,http://limonhost.net/,https://www.limonhost.net/,https://www.facebook.com/tr/?id=79621302384744...,GET,"[('urlencode', 'https://www.limonhost.net/'), ..."
812,http://clackesd.k12.or.us/,https://www.clackesd.org/,https://www.facebook.com/v14.0/plugins/page.ph...,GET,"[('www.clackesd.org',), ('urlencode', 'www.cla..."
848,http://clackesd.k12.or.us/,https://www.clackesd.org/,https://www.facebook.com/platform/plugin/tab/r...,GET,"[('www.clackesd.org',), ('urlencode', 'https:/..."
882,http://clackesd.k12.or.us/,https://www.clackesd.org/,https://www.facebook.com/login/?next=https%3A%...,GET,"[('www.clackesd.org',), ('urlencode', 'www.cla..."
...,...,...,...,...,...
79810,http://osheaga.com/,https://osheaga.com/fr,https://connect.facebook.net/signals/config/19...,GET,"[('urlencode', 'osheaga.com'), ('osheaga.com',)]"
79811,http://osheaga.com/,https://osheaga.com/fr,https://www.facebook.com/tr/?id=16057439283596...,GET,"[('urlencode', '/fr'), ('urlencode', 'https://..."
79818,http://osheaga.com/,https://osheaga.com/fr,https://connect.facebook.net/signals/config/38...,GET,"[('urlencode', 'osheaga.com'), ('osheaga.com',)]"
79819,http://osheaga.com/,https://osheaga.com/fr,https://www.facebook.com/tr/?id=19916247543847...,GET,"[('urlencode', '/fr'), ('urlencode', 'https://..."


### Tiktok

In [38]:
url_leak_print[url_leak_print['req_url'].str.contains('tiktok')] 

,init_url,final_url,req_url,req_method,url_leaks


### Leaks on Post Data

In [47]:
# exclude [] and empty string

post_leak = extractedDF[(extractedDF['post_leaks'] != '[]') & (extractedDF['post_leaks'] != 'None')]
post_leak = post_leak[post_leak['post_leaks'].notnull()]
post_leak

,init_url,final_url,req_url,req_method,ref_data,post_data,url_leaks,post_leaks,referer_leaks
431,http://farsondigitalwatercams.com/,https://www.farsondigitalwatercams.com/,https://maps.googleapis.com/$rpc/google.intern...,POST,https://www.farsondigitalwatercams.com/,"[[[43.25051468911406,-25.557122316294674],[63....",[],"[('www.farsondigitalwatercams.com',), ('https:...",[]
533,http://farsondigitalwatercams.com/,https://www.farsondigitalwatercams.com/,https://dc.services.visualstudio.com/v2/track,POST,https://www.farsondigitalwatercams.com/,"[{""time"":""2023-10-11T18:46:16.869Z"",""iKey"":""78...",[],"[('www.farsondigitalwatercams.com',), ('urlenc...",[]
557,http://farsondigitalwatercams.com/,https://www.farsondigitalwatercams.com/,https://bam.nr-data.net/events/1/9ac7216156?a=...,POST,https://www.farsondigitalwatercams.com/,"bel.7;1,c,,6bh,540,d6,'initialPageLoad,'https:...","[('www.farsondigitalwatercams.com',), ('urlenc...","[('www.farsondigitalwatercams.com',), ('https:...",[]
624,http://limonhost.net/,https://www.limonhost.net/,wss://socket.tidio.co/socket.io/?ppk=xezog98i6...,NaN,NaN,"420[""visitorRegister"",{""id"":""a412a907db674f33a...",[],"[('www.limonhost.net',), ('https://www.limonho...",[]
632,http://limonhost.net/,https://www.limonhost.net/,wss://ws.hotjar.com/api/v2/client/ws?v=4,NaN,NaN,"{""sid"":""20fb6f55-5bcf-4fb9-9805-f6ed7f975dda"",...",[],"[('www.limonhost.net',), ('https://www.limonho...",[]
...,...,...,...,...,...,...,...,...,...
79815,http://osheaga.com/,https://osheaga.com/fr,https://analytics.tiktok.com/api/v2/pixel,POST,https://osheaga.com/,"{""event"":""Pageview"",""message_id"":""messageId-16...",[],"[('https://osheaga.com/fr',), ('/fr',), ('oshe...",[]
79820,http://osheaga.com/,https://osheaga.com/fr,https://analytics.tiktok.com/api/v2/pixel/act,POST,https://osheaga.com/,"{""message_id"":""messageId-1697048461638-8852546...",[],"[('https://osheaga.com/fr',), ('/fr',), ('oshe...",[]
79821,http://osheaga.com/,https://osheaga.com/fr,https://www.youtube.com/youtubei/v1/log_event?...,POST,https://www.youtube.com/embed/Ku2hXbXvjrA,"{""context"":{""client"":{""hl"":""en"",""gl"":""US"",""cli...",[],"[('osheaga.com',)]",[]
79827,http://osheaga.com/,https://osheaga.com/fr,wss://ws.hotjar.com/api/v2/client/ws?v=4,NaN,NaN,"{""sid"":""6fe607cf-45ed-46ba-8956-f8be89ef6d7c"",...",[],"[('https://osheaga.com/fr',), ('/fr',), ('oshe...",[]


In [53]:
# drop unnecessary columns and print to csv

post_leak_print_raw =  post_leak.drop(['url_leaks', 'referer_leaks', 'ref_data'], axis=1)
post_leak_print =  post_leak.drop(['url_leaks', 'referer_leaks', 'ref_data'], axis=1)
post_leak_print['post_data'] = [x[:2048] for x in post_leak_print['post_data']] # truncate post_data to 2048 characters
post_leak_print.to_csv("post_print.csv", index=False)


## Leaks to Google, facebook, and Meta

In [56]:
# list of post data leaks to google
post_leak_print[post_leak_print['req_url'].str.contains('google')]

,init_url,final_url,req_url,req_method,post_data,post_leaks
431,http://farsondigitalwatercams.com/,https://www.farsondigitalwatercams.com/,https://maps.googleapis.com/$rpc/google.intern...,POST,"[[[43.25051468911406,-25.557122316294674],[63....","[('www.farsondigitalwatercams.com',), ('https:..."
2118,http://stacksocial.com/,https://www.stacksocial.com/,https://www.google-analytics.com/collect,POST,v=1&_v=j101&a=1881464239&t=event&ni=1&_s=1&dl=...,"[('www.stacksocial.com',), ('urlencode', 'http..."
2119,http://stacksocial.com/,https://www.stacksocial.com/,https://www.google-analytics.com/collect,POST,v=1&_v=j101&a=1881464239&t=event&ni=1&_s=1&dl=...,"[('www.stacksocial.com',), ('urlencode', 'http..."
2129,http://stacksocial.com/,https://www.stacksocial.com/,https://www.google-analytics.com/collect,POST,v=1&_v=j101&a=1881464239&t=event&ni=1&_s=1&dl=...,"[('www.stacksocial.com',), ('urlencode', 'http..."
2131,http://stacksocial.com/,https://www.stacksocial.com/,https://www.google-analytics.com/collect,POST,v=1&_v=j101&a=1881464239&t=event&ni=1&_s=1&dl=...,"[('www.stacksocial.com',), ('urlencode', 'http..."
2145,http://stacksocial.com/,https://www.stacksocial.com/,https://www.google-analytics.com/collect,POST,v=1&_v=j101&a=1881464239&t=event&ni=1&_s=1&dl=...,"[('www.stacksocial.com',), ('urlencode', 'http..."
8252,http://ticketportal.sk/,https://www.ticketportal.sk/,https://play.google.com/log?format=json&hasfas...,POST,"[[1,null,null,null,null,null,null,null,null,nu...","[('www.ticketportal.sk',)]"
10143,http://vhnd.com/,https://www.vhnd.com/,https://clients6.google.com/rpc?key=AIzaSyCKSb...,POST,"{""method"":""pos.plusones.get"",""id"":""p"",""params""...","[('www.vhnd.com',), ('https://www.vhnd.com/',)]"
17316,http://flakeads.co.uk/,https://flakeads.co.uk/,https://fundingchoicesmessages.google.com/el/A...,POST,"[null,null,null,null,null,null,null,null,16970...","[('flakeads.co.uk',)]"
17372,http://flakeads.co.uk/,https://flakeads.co.uk/,https://fundingchoicesmessages.google.com/el/A...,POST,"[null,null,null,null,null,null,null,null,16970...","[('flakeads.co.uk',)]"


In [58]:
# list of post data leaks to Meta or Facebook
post_leak_print[post_leak_print['req_url'].str.contains('meta') | post_leak_print['req_url'].str.contains('facebook')]

,init_url,final_url,req_url,req_method,post_data,post_leaks
849,http://clackesd.k12.or.us/,https://www.clackesd.org/,https://www.facebook.com/platform/plugin/page/...,POST,event_name=page_plugin.tab.configured&page_id=...,"[('www.clackesd.org',), ('urlencode', 'https:/..."
7384,http://decomyplace.com/,https://decomyplace.com/,https://www.facebook.com/platform/plugin/page/...,POST,event_name=page_plugin.tab.configured&page_id=...,"[('urlencode', 'https://decomyplace.com/'), ('..."
10141,http://vhnd.com/,https://www.vhnd.com/,https://sumo.com/apps/share/facebook/queue,POST,site_id=60e905548382947796284b87e360a9b424eea0...,"[('www.vhnd.com',), ('urlencode', 'https://www..."
10142,http://vhnd.com/,https://www.vhnd.com/,https://sumo.com/apps/share/facebook/queue,POST,site_id=60e905548382947796284b87e360a9b424eea0...,"[('www.vhnd.com',), ('urlencode', 'https://www..."
26535,http://pamperedchef.ca/,https://www.pamperedchef.ca/,https://graph.facebook.com/,POST,id=https%3A%2F%2Fwww.pamperedchef.ca%2F&scrape...,"[('www.pamperedchef.ca',), ('urlencode', 'http..."
37120,http://transformertable.com/,https://transformertable.com/,https://www.facebook.com/tr/,POST,id=534544930375443&ev=Microdata&dl=https%3A%2F...,"[('urlencode', 'https://transformertable.com/'..."
42729,http://nickiminajofficial.com/,https://www.nickiminajofficial.com/,https://www.facebook.com/tr/,POST,id=567318173708059&ev=Microdata&dl=https%3A%2F...,"[('urlencode', 'https://www.nickiminajofficial..."
62787,http://aismiley.co.jp/,https://aismiley.co.jp/,https://www.facebook.com/platform/plugin/page/...,POST,event_name=page_plugin.tab.configured&page_id=...,"[('aismiley.co.jp',), ('urlencode', 'https://a..."


In [59]:
# list of post data leaks to Meta or Facebook
post_leak_print[post_leak_print['req_url'].str.contains('tiktok')] 

,init_url,final_url,req_url,req_method,post_data,post_leaks
1496,http://yalla.deals/,https://yalla.deals/,https://analytics.tiktok.com/api/v2/pixel,POST,"{""event"":""Pageview"",""message_id"":""messageId-16...","[('yalla.deals',), ('https://yalla.deals/',)]"
1497,http://yalla.deals/,https://yalla.deals/,https://analytics.tiktok.com/api/v2/pixel,POST,"{""event"":""Browse"",""message_id"":""messageId-1697...","[('yalla.deals',), ('https://yalla.deals/',)]"
1513,http://yalla.deals/,https://yalla.deals/,https://analytics.tiktok.com/api/v2/pixel/act,POST,"{""message_id"":""messageId-1697052152218-8704384...","[('https://yalla.deals/',), ('yalla.deals',)]"
2124,http://stacksocial.com/,https://www.stacksocial.com/,https://analytics.tiktok.com/api/v2/pixel,POST,"{""event"":""Pageview"",""message_id"":""messageId-16...","[('www.stacksocial.com',), ('https://www.stack..."
2126,http://stacksocial.com/,https://www.stacksocial.com/,https://analytics.tiktok.com/api/v2/pixel/act,POST,"{""message_id"":""messageId-1697051915568-9051987...","[('www.stacksocial.com',), ('https://www.stack..."
19749,http://kudeta.com/,https://www.kudeta.com/,https://analytics.tiktok.com/api/v2/pixel,POST,"{""event"":""Pageview"",""message_id"":""messageId-16...","[('urlencode', 'https://www.kudeta.com/'), ('h..."
19755,http://kudeta.com/,https://www.kudeta.com/,https://analytics.tiktok.com/api/v2/pixel,POST,"{""event"":""Pageview"",""message_id"":""messageId-16...","[('urlencode', 'https://www.kudeta.com/'), ('h..."
19758,http://kudeta.com/,https://www.kudeta.com/,https://analytics.tiktok.com/api/v2/pixel/act,POST,"{""message_id"":""messageId-1697049843660-6108929...","[('urlencode', 'https://www.kudeta.com/'), ('h..."
19762,http://kudeta.com/,https://www.kudeta.com/,https://analytics.tiktok.com/api/v2/pixel/act,POST,"{""message_id"":""messageId-1697049843896-5771717...","[('urlencode', 'https://www.kudeta.com/'), ('h..."
21426,http://aupaircare.com/,https://www.aupaircare.com/,https://analytics.tiktok.com/api/v2/pixel,POST,"{""event"":""Pageview"",""message_id"":""messageId-16...","[('https://www.aupaircare.com/',), ('www.aupai..."
